In [ ]:
from datetime import datetime
import pandas as pd
from selenium import webdriver


week_str = '20221120'
# datetime object containing current date and time
now = datetime.now()


d_id_to_city = {'1078741': 'Chicago, IL', 
                'RunDMRC': 'Dallas, TX', 
                'lamrc': 'Los Angeles, CA',
                'nycmrc': 'New York, NY',
                '1020331': 'San Francisco, CA',
                'detroitmrc': 'Detroit, MI',
                'dcmuslimsrun': 'Washington, DC',
                '1088734': 'Houston, TX',
                'seattlemrc': 'Seattle, WA',
                'sdmrc': 'San Diego, CA',
                }

d_club_data = {}
for club in d_id_to_city:
    driver = webdriver.Chrome()
    location = d_id_to_city[club]
    driver.get('https://www.strava.com/clubs/' + club + '/leaderboard')
    
    dfs = pd.read_html(driver.page_source)
    
    d_club_data[location] = dfs[-1]
    print(location, len(dfs[-1]))
    d_club_data[location]['timestamp'] = now
    d_club_data[location]['location'] = location
    
    driver.close()


In [ ]:
from math import floor

# handle LA

def fixtime(t):
    if 'h' in t[0]:
        return (float(t[0][:-1]) * 60) + float(t[1][:-1])
    return float(t[0][:-1])

d_club_data['Los Angeles, CA']['time_hold'] = d_club_data['Los Angeles, CA']['Time'].map(lambda x: x.split(" "))
d_club_data['Los Angeles, CA']['time_hold'] = d_club_data['Los Angeles, CA']['time_hold'].map(lambda x: fixtime(x))
d_club_data['Los Angeles, CA']['dist_LA'] = d_club_data['Los Angeles, CA']['Distance'].map(lambda x: float(x[:-3]))
d_club_data['Los Angeles, CA'] = d_club_data['Los Angeles, CA'][d_club_data['Los Angeles, CA']['dist_LA'] > 0]
d_club_data['Los Angeles, CA']['Avg. Pace'] = d_club_data['Los Angeles, CA']['time_hold'] / d_club_data['Los Angeles, CA']['dist_LA']
d_club_data['Los Angeles, CA']['Avg. Pace'] = d_club_data['Los Angeles, CA']['Avg. Pace'].map(lambda x: str(floor(x)) + ":" + str(int((x - floor(x)) * 60)) + " /km")
d_club_data['Los Angeles, CA']

In [ ]:
df_all = pd.concat([d_club_data[k] for k in d_club_data])
df_all = df_all[df_all['Distance'] != 'There are no results.']
df_all['dist_float'] = df_all['Distance'].map(lambda x: float(x[:-3]))
df_all['avg_pace_float'] = df_all['Avg. Pace'].map(lambda x: x[:-4].split(":"))
df_all['avg_pace_km_min'] = df_all['avg_pace_float'].map(lambda x: float((60 * int(x[0])) + int(x[1])) / 60)
df_all['time_spent'] = df_all['avg_pace_km_min'] * df_all['dist_float']

df_all = df_all.sort_values(by=['dist_float'], ascending=False)
df_all['National Rank'] = [x for x in range(1, len(df_all) + 1)]
df_all['Location Rank'] = df_all['Rank']
df_all['elevation_gain'] = df_all['Elev. Gain']
df_all['elevation_gain'] = df_all['elevation_gain'].map(lambda x: ''.join(x.split(",")))
df_all['elevation_gain'] = df_all['elevation_gain'].map(lambda x: float(x[:-2]) if x != '--' else 0)
df_all.to_csv("rawdata/" + "raw_data_" + week_str + ".csv", index=False)

df_fts = df_all[['Athlete', 'dist_float', 'avg_pace_km_min', 'elevation_gain', 'time_spent', 'location']].copy(deep=True)

# dist rank
df_fts = df_fts.sort_values(by=['dist_float'], ascending=False)
df_fts['dist_rank'] = [x for x in range(1, len(df_fts) + 1)]

# avg_pace_km_min rank
df_fts = df_fts.sort_values(by=['avg_pace_km_min'], ascending=True)
df_fts['pace_rank'] = [x for x in range(1, len(df_fts) + 1)]

# elevation_gain rank
df_fts = df_fts.sort_values(by=['elevation_gain'], ascending=False)
df_fts['elevation_rank'] = [x for x in range(1, len(df_fts) + 1)]

df_all = df_all[['National Rank', 'Athlete', 'Distance', 'Runs', 'Longest', 'elevation_gain', 'timestamp', 'location', 'dist_float', 'Location Rank']]

In [ ]:
df_all.sample(5)

In [ ]:
df_fts['National Rank'] = (df_fts['dist_rank'] + df_fts['pace_rank'] + df_fts['elevation_rank']) / 3
df_fts = df_fts.sort_values(by=['National Rank'], ascending=True)
df_fts['National Rank'] = [x for x in range(1, len(df_fts) + 1)]
df_fts = df_fts[['National Rank', 'Athlete', 'dist_float', 'avg_pace_km_min', 'elevation_gain', 'time_spent', 'location']]
df_fts

In [ ]:
df_fts.groupby(['location']).sum('dist_float')

In [ ]:
df_fts.to_csv('MRC_week_' + week_str + '.csv', index=False)

In [ ]:
df_all.groupby(['location']).sum('dist_float')